In [1]:
!wget --no-cache -O init.py -q https://raw.githubusercontent.com/rramosp/2021.deeplearning/master/content/init.py
import init; init.init(force_download=False); init.get_weblink()

enabling encryption...
encryption enabled


ImportError: libpython3.7m.so.1.0: cannot open shared object file: No such file or directory